In [2]:
import torch
import numpy as np

[HAMI-core Msg(48810:139736112688000:libvgpu.c:836)]: Initializing.....


In [3]:
torch.__version__

'2.4.0+cu121'

### 基本数据处理与计算操作
#### 创建Tensor

In [28]:
# 创建一个2x3的未初始化的Tensor
x1 = torch.empty(2, 3)
# 创建一个2x3的随机初始化的Tensor
x2 = torch.rand(2,3)
# 创建一个2x3的long型全0的Tensor
x3 = torch.zeros(2, 3, dtype=torch.long)
# 为每一个元素以给定的mean和std用高斯分布生成随机数
x4 = torch.normal(mean=0.5, std=torch.arange(1., 6.))
# randn 服从N(0，1)的正态分布
x5 = torch.randn(2, 3)

x1, x2, x3, x4, x5

(tensor([[1.0825e-34, 0.0000e+00, 0.0000e+00],
         [1.4013e-45, 0.0000e+00, 0.0000e+00]]),
 tensor([[0.9814, 0.0874, 0.5718],
         [0.6293, 0.1877, 0.5226]]),
 tensor([[0, 0, 0],
         [0, 0, 0]]),
 tensor([ 1.5466, -3.7387, -2.4466,  3.9320,  0.3221]),
 tensor([[-1.7507, -0.7708, -0.1977],
         [ 1.7947,  2.4549, -0.1717]]))

In [32]:
# 根据不同数据类型创建
x1 = torch.tensor([[5,5,3], [2,2,5]])
x2 = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
x3 = torch.from_numpy(np.array([[1, 2, 3], [4, 5, 6]]))
x1, x2, x3

(tensor([[5, 5, 3],
         [2, 2, 5]]),
 tensor([[1, 2, 3],
         [4, 5, 6]]),
 tensor([[1, 2, 3],
         [4, 5, 6]]))

In [35]:
x = torch.tensor([[1,2,3], [4,5,6]])
# 返回的tensor默认具有相同的torch.dtype和torch.device 
x1 = x.new_ones(2, 3, dtype=torch.float64)
# 指定新的数据类型
x2 = torch.randn_like(x, dtype=torch.float, device='cuda')
x1, x2

(tensor([[1., 1., 1.],
         [1., 1., 1.]], dtype=torch.float64),
 tensor([[ 1.2797, -0.0880,  1.2151],
         [ 1.1600, -0.2267,  0.4583]], device='cuda:0'))

#### 操作张量

##### 算术操作

In [43]:
x = torch.ones(2, 3) 
y = torch.eye(2, 3)
# 加法形式1
s1 = x + y
# 加法形式2
s2 = torch.add(x, y)
# 加法形式3，inplace(原地操作)，原值修改
s3 = y.add_(x)

x, y, s1, s2, s3, y

(tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]),
 tensor([[2., 1., 1.],
         [1., 2., 1.]]))

##### 索引
索引出来的结果与原数据`共享内存`，也即修改一个，另一个会跟着修改。

In [45]:
x = torch.ones(2, 3, device='cuda') 
y = x[0, :]
y += 1

x, y, x[0, :]

(tensor([[2., 2., 2.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([2., 2., 2.], device='cuda:0'),
 tensor([2., 2., 2.], device='cuda:0'))

##### 切片 index_select
沿着指定维度对输入进行切片，取index中指定的相应项(index 为一个 LongTensor)，然后返回到一个新的张量， 返回的张量与原始张量 Tensor 有相同的维度(在指定轴上)。

注意： `返回的张量不与原始张量共享内存空间`。<br>

参数:<br>
input (Tensor)         – 输入张量<br>
dim (int)              – 索引的轴<br>
index (LongTensor)     – 包含索引下标的一维张量<br>
out (Tensor, optional) – 目标张量<br>

In [46]:
x = torch.randn(3, 4)
indices = torch.LongTensor([0, 2])
y = torch.index_select(x, 0, indices)
x, indices, y

(tensor([[ 0.1912,  0.1199, -0.0856, -1.8215],
         [ 0.2932,  0.9630,  0.3461, -0.7105],
         [-1.2408, -1.6029,  0.8402,  0.3389]]),
 tensor([0, 2]),
 tensor([[ 0.1912,  0.1199, -0.0856, -1.8215],
         [-1.2408, -1.6029,  0.8402,  0.3389]]))

##### 改变形状 view/reshape
1. `view()`<br>
注意view()**返回的新Tensor与源Tensor虽然可能有不同的size**，但`共享data`。<br>
即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的 观察角度，内部数据并未改变)
2. `reshape()` 和 `clone()`<br> 
Pytorch还提供了一个 reshape() 方法可以改变形状，但是此函数并不能保证返回的是其拷贝，所以不推荐使用。<br> 
我们推荐先用 clone() 创造一个副本然后再使用view()。

In [51]:
x = torch.ones(2, 4)
# 一个tensor必须是连续的contiguous()才能被查看。
# 一开始不加contiguous()，报 “view size is not compatible ... ” 错误
y1 = x.view(8)
y2 = x.view(-1, 8) # -1所指的维度可以根据其他维度的值推出来

x, y1, y2, x.size(), y1.size(), y2.size()

(tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.]]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([[1., 1., 1., 1., 1., 1., 1., 1.]]),
 torch.Size([2, 4]),
 torch.Size([8]),
 torch.Size([1, 8]))

In [52]:
x += 1
print(x)
print(y1) # 也加了1

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.]])
tensor([2., 2., 2., 2., 2., 2., 2., 2.])


In [53]:
# torch.view()会改变原始张量，但是很多情况下，我们希望原始张量和变换后的张量互相不影响。为为了使创建的张量和原始张量不共享内存，我们需要使用第二种方法torch.reshape()， 同样可以改变张量的形状，但是此函数并不能保证返回的是其拷贝值，所以官方不推荐使用。推荐的方法是我们先用 `clone()` 创造一个张量副本然后再使用 `torch.view()`进行函数维度变换。<br>
# 注：使用 clone() 还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源 Tensor 。 

x1 = x.clone()
y3 = x.view(-1)
x1 += 1

x1, y3

(tensor([[3., 3., 3., 3.],
         [3., 3., 3., 3.]]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2.]))

##### gather
`torch.gather(input, dim, index, out=None)`<br>
官方定义：沿给定轴dim，将输入索引张量index指定位置的值进行聚合。<br>
通俗理解：给定轴dim，在input中，根据index指定的下标，选择元素重组成一个新的tensor，最后输出的out与index的size是一样的。<br>

对一个3维张量，输出可以定义为<br>

```
out[i][j][k] = tensor[index[i][j][k]][j][k]] # dim=0
out[i][j][k] = tensor[i][index[i][j][k]][k]] # dim=1
out[i][j][k] = tensor[i][j][index[i][j][k]]] # dim=3
```

In [56]:
_input = torch.tensor([[0.9, 0, 0], [0, 0.6, 0],[0, 0.7, 0]])
index = torch.tensor([0, 1, 1])
_input, index, index.view(-1, 1)

(tensor([[0.9000, 0.0000, 0.0000],
         [0.0000, 0.6000, 0.0000],
         [0.0000, 0.7000, 0.0000]]),
 tensor([0, 1, 1]),
 tensor([[0],
         [1],
         [1]]))

In [30]:
_input.gather(dim=1, index=index.view(-1, 1))
# dim=1，表示的是在第二维度上操作。
# 在index中，[0]表示第一行对应元素的下标，即[0.9]
#           [1]表示第二行对应元素的下标，即[0.6]
#           [1]表示第三行对应元素的下标，即[0.7]


tensor([[0.9000],
        [0.6000],
        [0.7000]])

#### 广播机制
当我们对两个形状不同的Tensor按元素运算时，可能会触发`广播(broadcasting)机制`。 <br>
先适当复制元素使这两个Tensor形状相同后再按元素运算。

In [57]:
# x 中第一行的2个 元素被广播(复制)到了第二行和第三行
# y 中第一列的3个元素被广播(复制)到 了第二列
x = torch.arange(1, 3).view(1, 2)
y = torch.arange(1, 4).view(3, 1)
x, y, x+y

(tensor([[1, 2]]),
 tensor([[1],
         [2],
         [3]]),
 tensor([[2, 3],
         [3, 4],
         [4, 5]]))

#### Tensor和NumPy相互转换
+ `numpy()`和`from_numpy()` <br>
这两个函数所产生的的Tensor和NumPy中的数组共享相同的内存(所以他们之间的转换很快)，改变其中一个时另一个也会改变!
+ `torch.tensor()` <br>
进行**数据拷贝**，所以返回的Tensor和原来 的数据不再共享内存。

##### Tensor转NumPy数组

In [63]:
# Tensor转NumPy数组
x = torch.ones(3)
y = x.numpy()
x += 1
y += 1
print(x, y)

# NumPy数组转 Tensor
x = np.ones(3)
y = torch.from_numpy(x)
x += 1
y += 1
print(x, y)

tensor([3., 3., 3.]) [3. 3. 3.]
[3. 3. 3.] tensor([3., 3., 3.], dtype=torch.float64)


In [65]:
# 使用torch.tensor()将NumPy数组转换成Tensor(不再共享内存)
x = np.ones(3)
y = torch.tensor(x)
x += 1
print(x, y)

[2. 2. 2.] tensor([1., 1., 1.], dtype=torch.float64)


#### Tensor on GPU
+ `to()`<br>
将Tensor在CPU和GPU(需要硬件支持)之间相互移动。

#### 自动求梯度
+ `autograd 包`<br>
根据输入和**前向传播**过程自动构建计算图，并执行**反向传播**。
+ `Tensor`是autograd包的核心类<br>
将其属性`.requires_grad`设置为True，它将开始追踪在其上的所有操作(这样就可以利用链式法则进行梯度传播了)。<br>
完成计算后，可以调用`.backward()`来完成所有梯度计算。此Tensor的梯度将累积到.grad属性中。<br>
如果不想要被继续追踪，可以调用`.detach()`将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。<br>
此外，还可以用`with torch.no_grad()`将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算 可训练参数(requires_grad=True)的梯度。
+ `Function`是另外一个很重要的类。<br>
Tensor和Function互相结合就可以构建一个记录有整个计算过程的**有向无环图(DAG)**。<br>
每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。

##### Tensor

In [39]:
# 创建一个Tensor并设置requires_grad=True:
# x是直接创建的，所以它没有grad_fn
x = torch.ones(2, 2, requires_grad=True)
x, x.grad_fn 

(tensor([[1., 1.],
         [1., 1.]], requires_grad=True),
 None)

In [40]:
# y 是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn
y = x + 2
y, y.grad_fn

(tensor([[3., 3.],
         [3., 3.]], grad_fn=<AddBackward0>),
 <AddBackward0 at 0x13503b2e8>)

In [41]:
z = y * y * 3 
out = z.mean()
z, out

(tensor([[27., 27.],
         [27., 27.]], grad_fn=<MulBackward0>),
 tensor(27., grad_fn=<MeanBackward0>))

In [42]:
# 通过.requires_grad_()来用in-place的方式改变requires_grad属性
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
a.requires_grad

False

In [43]:
a.requires_grad_(True)
a.requires_grad

True

In [44]:
b = (a * a).sum()
b.grad_fn

##### 梯度

In [45]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
out.backward()  # 等价于 out.backward(torch.tensor(1.))
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

令out为o，因为
$$
O = \frac 1 4 \sum_i^4 z_i = \frac 1 4 \sum_i^4 3y_i^2 = \frac 1 4 \sum_i^4 3(x_i+2)^2
$$
所以<br>
$$
\frac{\partial{O}}{\partial{x_i}}|_{x_i=1} = \frac 9 2 = 4.5
$$

In [46]:
# 注意grad是累加的
out2 = x.sum()
out2.backward()      # 梯度未清零，累加梯度
x.grad

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])

In [47]:
out3 = x.sum()
x.grad.data.zero_()
out3.backward()      # 梯度清零后，x的梯度为1
x.grad

tensor([[1., 1.],
        [1., 1.]])

##### 反向传播
`y.backward()`<br>
+ 如果 $y$ 是**标量**，则不需要为backward()传入任何参数。<br>
+ 如果 $y$ 是**向量**，需要传入一个与y同形的Tensor。<br>
这样为了避免向量(甚至更高维张量) 对张量求导，因此转换成标量对张量进行求导。<br>
`不允许张量对张量求导`，只允许标量对张量求导，求导结果是和自变量同形的张量。<br>
必要时我们要把张量通过将所有张量的元素`加权求和`的方式转换为标量。<br>

假设 $𝒚$ 由自变量 $𝒙$ 计算而来，$𝒘$ 是和 $𝒚$ 同形的张量则 $𝐲. 𝐛𝐚𝐜𝐤𝐰𝐚𝐫𝐝(𝒘)$ 的含义是: <br>
先计算 $𝒍$ = 𝐭𝐨𝐫𝐜𝐡.𝐬𝐮𝐦($𝒚$ ∗ $𝒘$)，因此 𝒍 是一个标量，然后我们再求 𝒍 对自变量 𝒙 的导数。

In [48]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
x, y, z

(tensor([1., 2., 3., 4.], requires_grad=True),
 tensor([2., 4., 6., 8.], grad_fn=<MulBackward0>),
 tensor([[2., 4.],
         [6., 8.]], grad_fn=<ViewBackward>))

此时 𝒛 不是一个标量，所以在调用backward()时需要传入一个和 𝒛 同形的权重向量进行加权求和来得到一个标量。

In [49]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float) 
z.backward(v)
x.grad

tensor([2.0000, 0.2000, 0.0200, 0.0020])

##### .detach()和.data

https://zhuanlan.zhihu.com/p/67184419